In [1]:
import os
import random
from PIL import Image, ImageFilter
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, models, optimizers
from tensorflow.random import set_seed
from sklearn.model_selection import train_test_split

seed_value = 1
os.environ['PYTHONHASHSEED']=str(seed_value)

random.seed(seed_value)
np.random.seed(seed_value)
set_seed(seed_value)


print("tensorflow version is:", tf.__version__)


temp0 = os.listdir('/Users/anu/Desktop/Spring2022/BMED8813/project/BHI_Group3/data/even_slices/0/')
ct0 = ['/Users/anu/Desktop/Spring2022/BMED8813/project/BHI_Group3/data/even_slices/0/' + i for i in temp0]

temp1 = os.listdir('/Users/anu/Desktop/Spring2022/BMED8813/project/BHI_Group3/data/even_slices/1/')
ct1 = ['/Users/anu/Desktop/Spring2022/BMED8813/project/BHI_Group3/data/even_slices/1/' + i for i in temp1]

temp2 = os.listdir('/Users/anu/Desktop/Spring2022/BMED8813/project/BHI_Group3/data/even_slices/2/')
ct2 = ['/Users/anu/Desktop/Spring2022/BMED8813/project/BHI_Group3/data/even_slices/2/' + i for i in temp2]

temp3 = os.listdir('/Users/anu/Desktop/Spring2022/BMED8813/project/BHI_Group3/data/even_slices/3/')
ct3 = ['/Users/anu/Desktop/Spring2022/BMED8813/project/BHI_Group3/data/even_slices/3/' + i for i in temp3]


tensorflow version is: 2.7.0


In [2]:
print("Total class 0 images:", len(ct0))
print("Total class 1 images:", len(ct1))
print("Total class 2 images:", len(ct2))
print("Total class 3 images:", len(ct3))

Total class 0 images: 254
Total class 1 images: 685
Total class 2 images: 125
Total class 3 images: 125


In [ ]:
zero pad all the data under max number of slices
im = 0
for i in all_files:
    im = np.load(i)
    if im.shape[0] < np.max(all_slices):
        im = np.pad(im, ((int((np.max(all_slices) - im.shape[0])/2), 
                          int((np.max(all_slices) - im.shape[0])/2)), (0,0), (0,0)))
    if im.shape[0] % 2 == 1:
        im = np.pad(im, ((1,0), (0,0), (0,0)))
    np.save(os.path.join(os.getcwd(), 'data/even_slices', i[-16:]), im)
        
t = np.load('/Users/anu/Desktop/Spring2022/BMED8813/project/BHI_Group3/data/even_slices/study_0003.npy')

In [11]:
int(all_files[0][-16])

array(['/Users/anu/Desktop/Spring2022/BMED8813/project/BHI_Group3/data/studies/CT-0/study_0018.npy',
       '/Users/anu/Desktop/Spring2022/BMED8813/project/BHI_Group3/data/studies/CT-0/study_0024.npy',
       '/Users/anu/Desktop/Spring2022/BMED8813/project/BHI_Group3/data/studies/CT-0/study_0030.npy',
       ...,
       '/Users/anu/Desktop/Spring2022/BMED8813/project/BHI_Group3/data/studies/CT-3/study_1096.npy',
       '/Users/anu/Desktop/Spring2022/BMED8813/project/BHI_Group3/data/studies/CT-3/study_1107_rot5.npy',
       '/Users/anu/Desktop/Spring2022/BMED8813/project/BHI_Group3/data/studies/CT-3/study_1106_rot5.npy'],
      dtype='<U96')

In [12]:
all_files = np.concatenate((ct0, ct1, ct2, ct3),axis=0)
all_labels = [int(all_files[i][-16]) for i in range(len(all_files))]

# X = np.zeros((420, 72, 512, 512))

# for i in range(len(all_files)):
#     temp = np.load(all_files[i])
#     X[i] = temp
    
# y = np.asarray(all_labels)

# print(f'Shape of image array is {X.shape}, and shape of label array is {y.shape}.')

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8)

ValueError: invalid literal for int() with base 10: 'n'

In [3]:
# check that there's 72 slices per image
all_slices = [np.shape(np.load(i))[0] for i in all_files]
print(f'Maximum is {np.max(all_slices)}, minimum is {np.min(all_slices)}, median is {np.median(all_slices)}')
plt.hist(all_slices)

NameError: name 'all_files' is not defined

In [199]:
model = models.Sequential() 

model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(1, 72, 512, 512)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten()) 

model.add(layers.Dense(168, activation='relu'))
model.add(layers.Dense(4)) # final Dense layer has 4 neurons representing the 4 classes

### fill in more layers here ###

print(model.summary())

model.compile(optimizer='Adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=20, #batch_size=32,
                    validation_data=(X_test, y_test))

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 70, 510, 16)       73744     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 35, 255, 16)      0         
 g2D)                                                            
                                                                 
 flatten_10 (Flatten)        (None, 142800)            0         
                                                                 
 dense_20 (Dense)            (None, 168)               23990568  
                                                                 
 dense_21 (Dense)            (None, 4)                 676       
                                                                 
Total params: 24,064,988
Trainable params: 24,064,988
Non-trainable params: 0
_________________________________________

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type NoneType).